In [2]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

image = preprocess(Image.open("garbage classification/Garbage classification/glass/glass1.jpg")).unsqueeze(0).to(device)
text = clip.tokenize(["the meterial is glass", "the meterial is cardboard", "the meterial is metal", "the meterial is paper", "the meterial is plastic", "the meterial is unknow"]).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

print("Label probs:", probs)  # Label probs: [[0.6094  0.0851  0.0366  0.02254 0.2207  0.02594]]

Label probs: [[0.6094  0.0851  0.0366  0.02254 0.2207  0.02594]]


In [7]:
import torch
import clip
from PIL import Image
import os

# Setup
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)
materials = ["glass", "cardboard", "metal", "paper", "plastic"]
text_tokens = clip.tokenize([f"the material is {material}" for material in materials]).to(device)

def classify_image(image_path):
    image = preprocess(Image.open(image_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        logits_per_image, _ = model(image, text_tokens)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    return probs.squeeze()

# Iterate over folders and images
root_dir = '/home/mingwei/Desktop/CLIP/garbage classification/Garbage classification'
accuracy_dict = {material: {'correct': 0, 'total': 0} for material in materials}

for material in materials:
    directory = os.path.join(root_dir, material)
    for filename in os.listdir(directory):
        if filename.endswith('.jpg'):
            image_path = os.path.join(directory, filename)
            probs = classify_image(image_path)
            predicted_index = probs.argmax()
            predicted_material = materials[predicted_index]
            # Increment counts
            accuracy_dict[material]['total'] += 1
            if material == predicted_material:
                accuracy_dict[material]['correct'] += 1

print(accuracy_dict)

# Calculate and print accuracy for each material
for material, counts in accuracy_dict.items():
    if counts['total'] > 0:
        accuracy = counts['correct'] / counts['total']
        print(f"Accuracy for {material}: {accuracy:.2f}")


{'glass': {'correct': 323, 'total': 501}, 'cardboard': {'correct': 0, 'total': 0}, 'metal': {'correct': 0, 'total': 0}, 'paper': {'correct': 0, 'total': 0}, 'plastic': {'correct': 0, 'total': 0}}
{'glass': {'correct': 323, 'total': 501}, 'cardboard': {'correct': 352, 'total': 403}, 'metal': {'correct': 0, 'total': 0}, 'paper': {'correct': 0, 'total': 0}, 'plastic': {'correct': 0, 'total': 0}}
{'glass': {'correct': 323, 'total': 501}, 'cardboard': {'correct': 352, 'total': 403}, 'metal': {'correct': 181, 'total': 410}, 'paper': {'correct': 0, 'total': 0}, 'plastic': {'correct': 0, 'total': 0}}
{'glass': {'correct': 323, 'total': 501}, 'cardboard': {'correct': 352, 'total': 403}, 'metal': {'correct': 181, 'total': 410}, 'paper': {'correct': 448, 'total': 594}, 'plastic': {'correct': 0, 'total': 0}}
{'glass': {'correct': 323, 'total': 501}, 'cardboard': {'correct': 352, 'total': 403}, 'metal': {'correct': 181, 'total': 410}, 'paper': {'correct': 448, 'total': 594}, 'plastic': {'correct': 